In [32]:
%load_ext autoreload
%autoreload 2

import os
import sys
module_path = os.path.abspath(os.path.join('../../../src/'))
if module_path not in sys.path:
    sys.path.append(module_path)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Load data

In [9]:
FIXED_SAMPLE_RATE = 48000

In [10]:
pickle_path = "/nfs/students/summer-term-2020/project-4/data/dataset1/dataset_resampled/"
training = pickle.load(open(pickle_path + "training.p","rb"))
validation = pickle.load(open(pickle_path + "validation.p","rb"))

In [11]:
max_length = 0

for sample in training:
    max_length = max(max_length, sample['data'][0].shape[0])

for sample in validation:
    max_length = max(max_length, sample['data'][0].shape[0])
    
print(max_length)

481489


In [12]:
def prepareData(data):
    zero_padded_data = torch.zeros(max_length)
    zero_padded_data[:data.shape[0]] = torch.from_numpy(data)
    return zero_padded_data

In [13]:
training_dataset = [(prepareData(sample['data'][0]), 1)
                        if sample['binary_class']=='positive' else (prepareData(sample['data'][0]), 0) 
                        for sample in training]
validation_dataset = [(prepareData(sample['data'][0]), 1)
                        if sample['binary_class']=='positive' else (prepareData(sample['data'][0]), 0) 
                        for sample in validation]

# Attacks

In [336]:
from classification.models.DeepRecursiveCNN import DeepRecursiveCNN

model_state_dict_path = "/nfs/students/summer-term-2020/project-4/yan/models/best_model_state_dict.pt"
model = DeepRecursiveCNN()
model.load_state_dict(torch.load(model_state_dict_path))
model = model.eval()

## FGSM

In [227]:
from attacks.fgsm import FGSM

valid_loader = torch.utils.data.DataLoader(validation_dataset, batch_size=1, shuffle=False)
attack_parameters = {'epsilon': 0.2}

fgsm = FGSM(model, valid_loader, attack_parameters, early_stopping=20)
fgsm.attack()

  3%|▎         | 53/1687 [00:05<02:35, 10.48it/s]

Early stopping


In [ ]:
# fgsm.showAdversarialExample(sr=FIXED_SAMPLE_RATE, target_class=1)

In [228]:
fgsm.report()

Attack-Parameters:	{'epsilon': 0.2}
Early stopping: 	True (20)

Successfully attacked:	20
Total attacked: 	49
Total processed:	54

Success-Rate: 		0.41
Perturbed Accurracy: 	0.54


## PGD

In [221]:
from attacks.pgd import PGD

valid_loader = torch.utils.data.DataLoader(validation_dataset, batch_size=1, shuffle=False)
attack_parameters = {'epsilon': 0.5, 'num_iter': 30}

pgd = PGD(model, valid_loader, attack_parameters, early_stopping=20)
pgd.attack()

  2%|▏         | 40/1687 [01:09<47:23,  1.73s/it]

Early stopping


In [ ]:
# pgd.showAdversarialExample(sr=FIXED_SAMPLE_RATE, target_class=1)

In [222]:
pgd.report()

Attack-Parameters:	{'epsilon': 0.5, 'num_iter': 30}
Early stopping: 	True (20)

Successfully attacked:	20
Total attacked: 	37
Total processed:	41

Success-Rate: 		0.54
Perturbed Accurracy: 	0.41


## Volume attacks

In [287]:
from attacks.volume import VolumeAttack

valid_loader = torch.utils.data.DataLoader(validation_dataset, batch_size=1, shuffle=False)
attack_parameters = {'epsilon': 0.2, 'num_iter': 30}

vol = VolumeAttack(model, valid_loader, attack_parameters, early_stopping=1)
vol.attack()

  0%|          | 7/1687 [00:14<58:05,  2.07s/it]

Early stopping


In [282]:
# vol.showAdversarialExample(sr=FIXED_SAMPLE_RATE, target_class=1)

In [283]:
vol.report()

Attack-Parameters:	{'epsilon': 0.2, 'num_iter': 30}
Early stopping: 	True (1)

Successfully attacked:	1
Total attacked: 	8
Total processed:	8

Success-Rate: 		0.12
Perturbed Accurracy: 	0.88


## Interpolation attack

In [284]:
path_tum_sound = '/nfs/students/summer-term-2020/project-4/yan/tum.wav'
tum_sound,sr = librosa.load(path_tum_sound, sr=FIXED_SAMPLE_RATE)
padding = int((max_length - len(tum_sound))/2)
zero_padded_data = torch.zeros(max_length)
zero_padded_data[padding:padding+tum_sound.shape[0]] = torch.from_numpy(tum_sound)
tum_sound = zero_padded_data

In [295]:
from attacks.interpolation import InterpolationAttack

valid_loader = torch.utils.data.DataLoader(validation_dataset, batch_size=1, shuffle=False)
attack_parameters = {'epsilon': 0.2, 'num_iter': 30, 'overlay_sound': tum_sound}

ia = InterpolationAttack(model, valid_loader, attack_parameters, early_stopping=1)
ia.attack()

  4%|▍         | 74/1687 [02:04<45:03,  1.68s/it]

Early stopping


In [298]:
# ia.showAdversarialExample(sr=FIXED_SAMPLE_RATE, target_class=1)

In [297]:
ia.report()

Attack-Parameters:	{'epsilon': 0.2, 'num_iter': 30, 'overlay_sound': tensor([0., 0., 0.,  ..., 0., 0., 0.])}
Early stopping: 	True (1)

Successfully attacked:	1
Total attacked: 	67
Total processed:	75

Success-Rate: 		0.01
Perturbed Accurracy: 	0.88


## Time stretching attack

In [332]:
from attacks.speed import TimeStretchAttack

valid_loader = torch.utils.data.DataLoader(validation_dataset, batch_size=1, shuffle=False)
attack_parameters = {'epsilon': 0.2, 'num_iter': 30}

tsa = TimeStretchAttack(model, valid_loader, attack_parameters, early_stopping=1)
tsa.attack()

  0%|          | 7/1687 [00:40<2:42:50,  5.82s/it]

Early stopping


In [334]:
# tsa.showAdversarialExample(sr=FIXED_SAMPLE_RATE, target_class=1)

In [335]:
tsa.report()

Attack-Parameters:	{'epsilon': 0.2, 'num_iter': 30}
Early stopping: 	True (1)

Successfully attacked:	1
Total attacked: 	8
Total processed:	8

Success-Rate: 		0.12
Perturbed Accurracy: 	0.88
